# Feature Engineering

In this notebook we're going to generate features for our link prediction classifier.

In [1]:
from neo4j import GraphDatabase

import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

plt.style.use('fivethirtyeight')
pd.set_option('display.float_format', lambda x: '%.3f' % x)

# tag::imports[]
from sklearn.ensemble import RandomForestClassifier
# end::imports[]

In [2]:
bolt_uri = "bolt://localhost:7687"
# bolt_uri = "bolt://link-prediction-neo4j"
driver = GraphDatabase.driver(bolt_uri, auth=("neo4j", "letmein"))
#driver = GraphDatabase.driver(bolt_uri, auth=("neo4j", "admin"), max_connection_lifetime=50000)

print(driver.address)

localhost:7687


We can create our classifier with the following code:

In [3]:
# Load the CSV files saved in the train/test notebook

df_train_under = pd.read_csv("data/df_train_under.csv")
df_test_under = pd.read_csv("data/df_test_under.csv")

In [4]:
df_train_under.sample(5)

,node1,node2,label
111375,71363,89757,1
144295,204877,204878,1
2474,213913,117370,0
106368,77404,77405,1
93835,43344,43346,1


In [5]:
df_test_under.sample(5)

,node1,node2,label
110017,135213,179064,1
71432,58387,170127,0
41531,260507,107342,0
131697,65514,235309,1
50269,191600,258596,0


# Generating graphy features

We’ll start by creating a simple model that tries to predict whether two authors will have a future collaboration based on features extracted from common authors, preferential attachment, and the total union of neighbors.

The following function computes each of these measures for pairs of nodes:

#### Link Prediction algorithms

Common neighbors captures the idea that two strangers who have a friend in common are more likely to be introduced than those who don’t have any friends in common.

Preferential Attachment is a measure used to compute the closeness of nodes, based on their shared neighbors.

Total Neighbors computes the closeness of nodes, based on the number of unique neighbors that they have. It is based on the idea that the more connected a node is, the more likely it is to receive new links.



In [14]:
# tag::graphy-features[]
def apply_graphy_features(data, rel_type):
    query = """
    UNWIND $pairs AS pair
    MATCH (p1) WHERE id(p1) = pair.node1
    MATCH (p2) WHERE id(p2) = pair.node2
    RETURN pair.node1 AS node1,
           pair.node2 AS node2,
           gds.alpha.linkprediction.commonNeighbors(p1, p2, {
             relationshipQuery: $relType}) AS cn,
           gds.alpha.linkprediction.preferentialAttachment(p1, p2, {
             relationshipQuery: $relType}) AS pa,
           gds.alpha.linkprediction.totalNeighbors(p1, p2, {
             relationshipQuery: $relType}) AS tn
    """
    pairs = [{"node1": node1, "node2": node2}  for node1,node2 in data[["node1", "node2"]].values.tolist()]
    
    with driver.session(database="neo4j") as session:
        result = session.run(query, {"pairs": pairs, "relType": rel_type})
        features = pd.DataFrame([dict(record) for record in result])    
    return pd.merge(data, features, on = ["node1", "node2"])
# end::graphy-features[]

Let's apply the function to our training DataFrame:

In [15]:
# tag::apply-graphy-features[]
df_train_under = apply_graphy_features(df_train_under, "CO_AUTHOR_EARLY")
df_test_under = apply_graphy_features(df_test_under, "CO_AUTHOR")
# end::apply-graphy-features[]

Now we're going to add some new features that are generated from graph algorithms.

## Triangles and The Clustering Coefficient

We'll start by running the [triangle count](clusteringCoefficientProperty) algorithm over our test and train sub graphs. This algorithm will return the number of triangles that each node forms, as well as each node's clustering coefficient. The clustering coefficient of a node indicates the likelihood that its neighbours are also connected.

### Community detection algorithms
The Triangle Count algorithm counts the number of triangles for each node in the graph. A triangle is a set of three nodes where each node has a relationship to the other two. In graph theory terminology, this is sometimes referred to as a 3-clique. The Triangle Count algorithm in the GDS library only finds triangles in undirected graphs.

The Local Clustering Coefficient algorithm computes the local clustering coefficient for each node in the graph. The local clustering coefficient Cn of a node n describes the likelihood that the neighbours of n are also connected. To compute Cn we use the number of triangles a node is a part of Tn, and the degree of the node dn.

In [19]:
# tag::train-triangles[]
query = """
CALL gds.triangleCount.write({
  nodeProjection: 'Author',
  relationshipProjection: {
    CO_AUTHOR_EARLY: {
      type: 'CO_AUTHOR_EARLY',
      orientation: 'UNDIRECTED'
    }
  },
  writeProperty: 'trianglesTrain'
});
"""

with driver.session(database="neo4j") as session:
    result = session.run(query)
# end::train-triangles[]    
    df = pd.DataFrame([dict(record) for record in result])
df

,writeMillis,nodePropertiesWritten,globalTriangleCount,nodeCount,postProcessingMillis,createMillis,computeMillis,configuration
0,607,80299,97205,80299,0,256,93,"{'writeConcurrency': 4, 'writeProperty': 'tria..."


In [20]:
# tag::test-triangles[]
query = """
CALL gds.triangleCount.write({
  nodeProjection: 'Author',
  relationshipProjection: {
    CO_AUTHOR: {
      type: 'CO_AUTHOR',
      orientation: 'UNDIRECTED'
    }
  },
  writeProperty: 'trianglesTest'
});
"""

with driver.session(database="neo4j") as session:
    result = session.run(query)
# end::test-triangles[]    
    df = pd.DataFrame([dict(record) for record in result])
df    

,writeMillis,nodePropertiesWritten,globalTriangleCount,nodeCount,postProcessingMillis,createMillis,computeMillis,configuration
0,423,80299,199813,80299,0,103,27,"{'writeConcurrency': 4, 'writeProperty': 'tria..."


In [21]:
# tag::train-coefficient[]
query = """
CALL gds.localClusteringCoefficient.write({
  nodeProjection: 'Author',
  relationshipProjection: {
    CO_AUTHOR_EARLY: {
      type: 'CO_AUTHOR',
      orientation: 'UNDIRECTED'
    }
  },
  writeProperty: 'coefficientTest'
});
"""

with driver.session(database="neo4j") as session:
    result = session.run(query)
# end::train-coefficient[]
    df = pd.DataFrame([dict(record) for record in result])
df

,writeMillis,nodePropertiesWritten,averageClusteringCoefficient,nodeCount,postProcessingMillis,createMillis,computeMillis,configuration
0,435,80299,0.657,80299,0,74,79,"{'writeConcurrency': 4, 'triangleCountProperty..."


In [37]:
# tag::test-coefficient[]
query = """
CALL gds.localClusteringCoefficient.write({
  nodeProjection: 'Author',
  relationshipProjection: {
    CO_AUTHOR_EARLY: {
      type: 'CO_AUTHOR',
      orientation: 'UNDIRECTED'
    }
  },
  writeProperty: 'coefficientTest'
});
"""

with driver.session(database="neo4j") as session:
    result = session.run(query)
# end::test-coefficient[]
    df = pd.DataFrame([dict(record) for record in result])
df

,nodePropertiesWritten,writeMillis,averageClusteringCoefficient,nodeCount,createMillis,computeMillis,configuration
0,80299,505,0.657,80299,52,84,"{'writeConcurrency': 4, 'triangleCountProperty..."


The following function will add these features to our train and test DataFrames:

In [38]:
# tag::triangles-coefficient-features[]
def apply_triangles_features(data, triangles_prop, coefficient_prop):
    query = """
    UNWIND $pairs AS pair
    MATCH (p1) WHERE id(p1) = pair.node1
    MATCH (p2) WHERE id(p2) = pair.node2
    RETURN pair.node1 AS node1,
    pair.node2 AS node2,
    apoc.coll.min([p1[$trianglesProp], p2[$trianglesProp]]) AS minTriangles,
    apoc.coll.max([p1[$trianglesProp], p2[$trianglesProp]]) AS maxTriangles,
    apoc.coll.min([p1[$coefficientProp], p2[$coefficientProp]]) AS minCoefficient,
    apoc.coll.max([p1[$coefficientProp], p2[$coefficientProp]]) AS maxCoefficient
    """    
    pairs = [{"node1": node1, "node2": node2}  for node1,node2 in data[["node1", "node2"]].values.tolist()]    
    params = {
    "pairs": pairs,
    "trianglesProp": triangles_prop,
    "coefficientProp": coefficient_prop
    }

    with driver.session(database="neo4j") as session:
        result = session.run(query, params)
        features = pd.DataFrame([dict(record) for record in result])       
    
    return pd.merge(data, features, on = ["node1", "node2"])
# end::triangles-coefficient-features[]

In [39]:
# tag::apply-triangles-coefficient-features[]
df_train_under = apply_triangles_features(df_train_under, "trianglesTrain", "coefficientTrain")
df_test_under = apply_triangles_features(df_test_under, "trianglesTest", "coefficientTest")
# end::apply-triangles-coefficient-features[]

In [40]:
df_train_under.sample(5)

,node1,node2,label,cn,pa,tn,minTriangles,maxTriangles,minCoefficient,maxCoefficient
2425,72616,72616,0,0.000,81.000,9.000,36,36,None,None
132535,145839,145841,1,3.000,48.000,13.000,6,22,None,None
98997,1774,52485,1,1.000,8.000,5.000,1,2,None,None
136311,170086,170087,1,1.000,4.000,3.000,1,1,None,None
823,36585,205312,0,0.000,18.000,9.000,3,5,None,None


In [41]:
df_test_under.sample(5)

,node1,node2,label,cn,pa,tn,minTriangles,maxTriangles,minCoefficient,maxCoefficient
48953,11622,52568,0,0.000,88.000,19.000,9,18,0.321,0.327
102328,156250,156251,1,2.000,24.000,8.000,4,9,0.600,0.667
42921,166420,27712,0,1.000,125.000,29.000,2,254,0.200,0.847
41298,145373,238345,0,0.000,24.000,10.000,5,6,0.333,1.000
57149,18590,111823,0,0.000,18.000,19.000,0,25,0.000,0.163


In [ ]:
# tag::train-lpa[]
query = """
CALL gds.labelPropagation: {
    CO_AUTHOR_EARLY: {
      type: 'CO_AUTHOR_EARLY',
      orientation: 'UNDIRECTED'
    }
  },
  writeProperty: "partitionTrain"
});opagation.write({
  nodeProjection: "Author",
  relationshipProjection: {
    CO_AUTHOR_EARLY: {
      type: 'CO_AUTHOR_EARLY',
      orientation: 'UNDIRECTED'
    }
  },
  writeProperty: "partitionTrain"
});
"""

with driver.session(database="neo4j") as session:
    result = session.run(query)
# end::train-lpa[]
    df = pd.DataFrame([dict(record) for record in result])
df   

## Community Detection

Community detection algorithms evaluate how a group is clustered or partitioned. Nodes are considered more similar to nodes that fall in their community than to nodes in other communities.

We'll run two community detection algorithms over the train and test sub graphs - Label Propagation and Louvain. First up, Label Propagation: 

In [46]:
# tag::train-lpa[]
query = """
CALL gds.labelPropagation.write({
  nodeProjection: "Author",
  relationshipProjection: {
    CO_AUTHOR_EARLY: {
      type: 'CO_AUTHOR_EARLY',
      orientation: 'UNDIRECTED'
    }
  },
  writeProperty: "partitionTrain"
});
"""

with driver.session(database="neo4j") as session:
    result = session.run(query)
# end::train-lpa[]
    df = pd.DataFrame([dict(record) for record in result])
df    

,nodePropertiesWritten,createMillis,computeMillis,writeMillis,postProcessingMillis,communityCount,ranIterations,didConverge,communityDistribution,configuration
0,80299,22,90,457,19,43781,8,True,"{'p99': 15, 'min': 1, 'max': 175, 'mean': 1.83...","{'maxIterations': 10, 'writeConcurrency': 4, '..."


In [45]:
# tag::test-lpa[]
query = """
CALL gds.labelPropagation.write({
  nodeProjection: "Author",
  relationshipProjection: {
    CO_AUTHOR: {
      type: 'CO_AUTHOR',
      orientation: 'UNDIRECTED'
    }
  },
  writeProperty: "partitionTest"
});
"""

with driver.session(database="neo4j") as session:
    result = session.run(query)
# end::test-lpa[]    
    df = pd.DataFrame([dict(record) for record in result])
df    

,nodePropertiesWritten,createMillis,computeMillis,writeMillis,postProcessingMillis,communityCount,ranIterations,didConverge,communityDistribution,configuration
0,80299,46,224,465,28,15822,9,True,"{'p99': 42, 'min': 1, 'max': 440, 'mean': 5.07...","{'maxIterations': 10, 'writeConcurrency': 4, '..."


And now Louvain. The Louvain algorithm returns intermediate communities, which are useful for finding fine grained communities that exist in a graph. We'll add a property to each node containing the community revealed on the first iteration of the algorithm:

In [47]:
# tag::train-louvain[]
query = """
CALL gds.louvain.stream({
  nodeProjection: 'Author',
  relationshipProjection: {
    CO_AUTHOR_EARLY: {
      type: 'CO_AUTHOR_EARLY',
      orientation: 'UNDIRECTED'
    }
  },
  includeIntermediateCommunities: true
})
YIELD nodeId, communityId, intermediateCommunityIds
WITH gds.util.asNode(nodeId) AS node, intermediateCommunityIds[0] AS smallestCommunity
SET node.louvainTrain = smallestCommunity;
"""

with driver.session(database="neo4j") as session:
    display(session.run(query).consume().counters)
# end::train-louvain[]    

{'properties_set': 80299}

In [48]:
# tag::test-louvain[]
query = """
CALL gds.louvain.stream({
  nodeProjection: 'Author',
  relationshipProjection: {
    CO_AUTHOR: {
      type: 'CO_AUTHOR',
      orientation: 'UNDIRECTED'
    }
  },
  includeIntermediateCommunities: true
})
YIELD nodeId, communityId, intermediateCommunityIds
WITH gds.util.asNode(nodeId) AS node, intermediateCommunityIds[0] AS smallestCommunity
SET node.louvainTest = smallestCommunity;
"""

with driver.session(database="neo4j") as session:
    display(session.run(query).consume().counters)
# end::test-louvain[]    

{'properties_set': 80299}

In [49]:
# tag::community-features[]
def apply_community_features(data, partition_prop, louvain_prop):
    query = """
    UNWIND $pairs AS pair
    MATCH (p1) WHERE id(p1) = pair.node1
    MATCH (p2) WHERE id(p2) = pair.node2
    RETURN pair.node1 AS node1,
    pair.node2 AS node2,
    gds.alpha.linkprediction.sameCommunity(p1, p2, $partitionProp) AS sp,    
    gds.alpha.linkprediction.sameCommunity(p1, p2, $louvainProp) AS sl
    """
    pairs = [{"node1": node1, "node2": node2}  for node1,node2 in data[["node1", "node2"]].values.tolist()]
    params = {
    "pairs": pairs,
    "partitionProp": partition_prop,
    "louvainProp": louvain_prop
    }
    
    with driver.session(database="neo4j") as session:
        result = session.run(query, params)
        features = pd.DataFrame([dict(record) for record in result])
    
    return pd.merge(data, features, on = ["node1", "node2"])
# end::community-features[]

In [50]:
# tag::apply-community-features[]
df_train_under = apply_community_features(df_train_under, "partitionTrain", "louvainTrain")
df_test_under = apply_community_features(df_test_under, "partitionTest", "louvainTest")
# end::apply-community-features[]

In [51]:
# tag::train-after-features[]
df_train_under.drop(columns=["node1", "node2"]).sample(5, random_state=42)
# end::train-after-features[]

,label,cn,pa,tn,minTriangles,maxTriangles,minCoefficient,maxCoefficient,sp,sl
116159,1,4.000,25.000,6.000,10,10,None,None,1.000,1.000
107109,1,2.000,12.000,5.000,3,3,None,None,1.000,1.000
82258,1,2.000,9.000,4.000,3,3,None,None,1.000,1.000
79773,0,0.000,15.000,8.000,3,10,None,None,1.000,0.000
42367,0,0.000,8.000,6.000,1,6,None,None,1.000,0.000


In [52]:
# tag::test-after-features[]
df_test_under.drop(columns=["node1", "node2"]).sample(5, random_state=42)
# end::test-after-features[]

,label,cn,pa,tn,minTriangles,maxTriangles,minCoefficient,maxCoefficient,sp,sl
35910,0,0.000,48.000,14.000,15,28,1.000,1.000,0.000,0.000
98339,1,3.000,665.000,51.000,20,70,0.117,0.118,0.000,0.000
147201,1,1.000,6.000,4.000,1,2,0.667,1.000,1.000,1.000
126570,1,4.000,190.000,25.000,20,42,0.246,0.444,0.000,0.000
124563,1,2.000,15.000,6.000,2,3,0.300,0.667,1.000,1.000


In [54]:
# Re-order so that label is last
df_train_under = df_train_under.reindex(columns=sorted(df_train_under.columns))
df_train_under = df_train_under.reindex(columns=(list([a for a in df_train_under.columns if a != 'label']) + ['label'] ))

df_test_under = df_test_under.reindex(columns=sorted(df_test_under.columns))
df_test_under = df_test_under.reindex(columns=(list([a for a in df_test_under.columns if a != 'label']) + ['label'] ))


# Save our DataFrames to CSV files for use in the next notebook

df_train_under.to_csv("data/df_train_under_all.csv", index=False)
df_test_under.to_csv("data/df_test_under_all.csv", index=False)

# df_train_under = pd.read_csv("data/df_train_under_all.csv")
# df_test_under = pd.read_csv("data/df_test_under_all.csv")

# Save the samples as CSV files as well
df_train_under.drop(columns=["node1", "node2"]).sample(5, random_state=42).to_csv("data/df_train_under_sample.csv", index=False, float_format='%g')
df_test_under.drop(columns=["node1", "node2"]).sample(5, random_state=42).to_csv("data/df_test_under_sample.csv", index=False, float_format='%g')